In [3]:
# Import required libraries
import pandas as pd
import numpy as np
import dash
#import dash_html_components as html
import dash_html_components as html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [5]:
filtered_df = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
filtered_df

,Launch Site,class
0,CCAFS LC-40,7
1,CCAFS SLC-40,3
2,KSC LC-39A,10
3,VAFB SLC-4E,4


In [6]:
spacex_df.columns

Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')

In [7]:
# Task 1 Dropdown List
launch_list = []
launch_list.append({'label': 'All Sites', 'value': 'ALL'})
for site in spacex_df['Launch Site'].unique():
    launch_list.append({'label': site, 'value': site})
    
launch_list

[{'label': 'All Sites', 'value': 'ALL'},
 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}]

In [8]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(
    children=[
        html.H1('SpaceX Launch Records Dashboard',
                style={'textAlign': 'center', 
                       'color': '#503D36', 
                       'font-size': 40}),
        
    #####################################################################
    # TASK 1: Add a dropdown list to enable Launch Site selection
    # The default select value is for ALL sites
    # dcc.Dropdown(id='site-dropdown',...)
    #####################################################################
        html.Br(),
        dcc.Dropdown(
            id = 'site-dropdown',
            options = launch_list, 
            value = 'ALL',
            placeholder = "Select a Launch Site here",
            searchable = True
        ),
        
    ###########################################################################################
    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    # If a specific launch site was selected, show the Success vs. Failed counts for the site
    ###########################################################################################

        html.Div(dcc.Graph(id='success-pie-chart')),
        html.Br(),

        html.P("Payload range (Kg):"),
    ############################################### 
    # TASK 3: Add a slider to select payload range
    # dcc.RangeSlider(id='payload-slider',...)
    ###############################################  
        dcc.RangeSlider(
            id = 'payload-slider',
            min=0, max=10000, step=1000,
            marks = {str(int(i)): int(i) for i in np.linspace(0, 10000, 11)},
            value = [min_payload, max_payload]
        ),
        
    ########################################################################################
    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    ########################################################################################


        # TASK 4: Add a scatter chart to show the correlation between payload and launch success
        html.Div(dcc.Graph(id='success-payload-scatter-chart')),
        ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    site_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='count')
    
    if entered_site == 'ALL':
        fig = px.pie(
            spacex_df, 
            values='class', 
            names='Launch Site', 
            title='Total Success Launches for Sites'
        )
        return fig
    else:
        fig = px.pie(
            site_df, 
            values = 'count', 
            names = 'class', 
            title = f'Total Success Launches for Site {entered_site}'
        )
        return fig
        # return the outcomes piechart for a selected site





# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")])
def update_output_container(entered_site, payload_slide):
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    low, high = payload_slide
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    mask_filter = (filtered_df['Payload Mass (kg)'] >= low) & (filtered_df['Payload Mass (kg)'] <= high)

                            
    if entered_site == 'ALL':
        fig = px.scatter(
            spacex_df[mask], 
            x='Payload Mass (kg)', 
            y='class', 
            color="Booster Version Category",
            title='Correlation between Payload and Success for all Sites in Payload Range {} kg - {} kg'.format(int(low),int(high))
        )
        return fig
    else:
        fig = px.scatter(
            filtered_df[mask_filter],  
            x='Payload Mass (kg)', 
            y='class', 
            color="Booster Version Category",
            title='Correlation between Payload and Success for site {} in Payload Range {} kg - {} kg'.format(entered_site,int(low),int(high))
        )
        return fig
            
    
    
    

# Run the app
if __name__ == '__main__':
    app.run_server()